# 快速入门指南
本教程将简要介绍如何使用 LangChain 构建端到端语言模型应用程序。

## 安装
首先，使用以下命令安装 LangChain: 
```python
pip install langchain
conda install langchain -c conda-forge
pip install langchain-openai
```

## 环境设定
使用 LangChain 通常需要与一个或多个模型提供程序、数据存储、 API 等集成。
```
OPENAI_API_KEY="..."
```

这里我们使用python-dotenv来加载环境变量。

In [50]:
import os
from dotenv import load_dotenv

# 加载.env文件中的环境变量
load_dotenv()

True

## LLM: 从语言模型中获取预测
LangChain 最基本的构建块是对某些输入调用 LLM。

让我们来看一个简单的例子。

我们假设我们正在构建一个基于公司产品生成公司名称的服务。

为此，我们首先需要导入 LLM 包装器。

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-3.5-turbo',
)

In [4]:
text = "how can langsmith help with testing?"
llm.invoke(text)

AIMessage(content='Langsmith can help with testing in various ways, including:\n\n1. Writing test cases: Langsmith can assist in writing thorough and comprehensive test cases to ensure that all aspects of the software are tested.\n\n2. Test automation: Langsmith can help automate test scripts to streamline the testing process and increase efficiency.\n\n3. Test execution: Langsmith can execute test cases and report any bugs or issues that are identified during testing.\n\n4. Performance testing: Langsmith can conduct performance testing to ensure that the software meets performance requirements and can handle a certain number of users or transactions.\n\n5. Security testing: Langsmith can help with security testing to identify and address any vulnerabilities in the software.\n\n6. Regression testing: Langsmith can assist in conducting regression testing to ensure that new changes or updates to the software do not negatively impact existing functionality.\n\nOverall, Langsmith can provi

如果您不想设置环境变量，可以api_key在启动 OpenAI LLM 类时直接通过命名参数传递密钥：


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'),
                model='gpt-3.5-turbo',
                base_url=os.getenv('OPENAI_API_BASE'))

In [8]:
llm.invoke(text)

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to efficiently and effectively test software applications. Langsmith can also provide testing services, such as quality assurance testing and performance testing, to ensure that software applications are thoroughly tested and free of defects before deployment. Additionally, Langsmith can offer expertise in test automation, test case design, and test strategy development to help organizations improve their testing processes and achieve better testing outcomes.', response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 15, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4485a85-8bf1-470a-a3a3-0367b422e8be-0')

## 提示模板
我们还可以通过提示模板来指导其响应。提示模板将原始用户输入转换为更好的 LLM 输入。

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [13]:
# 我们现在可以将它们组合成一个简单的 LLM 链：
chain = prompt | llm

In [19]:
chain.invoke({"input": text})

AIMessage(content='Langsmith is a powerful tool that can significantly help with testing by facilitating automated testing and enabling efficient management of test cases. Here are some ways Langsmith can assist with testing:\n\n1. Automated Testing: Langsmith allows you to automate the testing process by writing scripts and test cases in a structured manner. This can help in executing tests quickly and accurately, reducing the manual effort required for testing.\n\n2. Test Case Management: Langsmith provides a centralized platform to create, store, and manage test cases effectively. You can organize test cases based on different criteria such as test type, priority, and status, making it easier to track and maintain test coverage.\n\n3. Reusability: Langsmith supports the reuse of test cases and scripts, enabling you to save time and effort by leveraging existing test assets across different testing scenarios. This can improve efficiency and consistency in testing practices.\n\n4. Col

ChatModel（因此，该链）的输出是一条消息。然而，使用字符串通常要方便得多。让我们添加一个简单的输出解析器来将聊天消息转换为字符串。



In [20]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [21]:
# 我们现在可以将其添加到之前的链中：
chain = prompt | llm | output_parser

In [22]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a powerful tool for testing various aspects of your software application. Here are some ways Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith can be used to write automated test scripts that can simulate user interactions with your application. This can help you ensure that your application functions correctly and consistently across different scenarios.\n\n2. **Performance Testing**: Langsmith can be used to simulate a large number of virtual users accessing your application simultaneously, allowing you to test the performance and scalability of your application under various load conditions.\n\n3. **Integration Testing**: Langsmith can be used to automate the testing of integrations between different components of your application, ensuring that they work together as expected.\n\n4. **Regression Testing**: Langsmith can be used to automate regression testing, allowing you to quickly verify that new code changes have not introduced any unintend

## 检索链

为了正确回答最初的问题（“langsmith 如何帮助测试？”），我们需要为法学硕士提供额外的背景信息。我们可以通过检索来做到这一点。当您有太多数据无法直接传递给 LLM时，检索非常有用。然后，您可以使用检索器仅获取最相关的部分并将其传递进去。

在此过程中，我们将从检索器中查找相关文档，然后将它们传递到提示符中。检索器可以由任何东西支持 - SQL 表、互联网等 - 但在本例中，我们将填充向量存储并将其用作检索器。有关矢量存储的更多信息，请参阅此文档。

首先，我们需要加载要索引的数据。为此，我们将使用 WebBaseLoader。这需要安装BeautifulSoup：
`pip install beautifulsoup4`

In [23]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [38]:
docs[0].page_content

"\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we 

接下来，我们需要将其索引到向量存储中。这需要一些组件，即嵌入模型和向量存储。

In [25]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

现在，我们可以使用此嵌入模型将文档提取到向量存储中。为了简单起见，我们将使用一个简单的本地向量库[FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/) 。

首先我们需要安装所需的软件包：
`pip install faiss-cpu`

In [26]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

现在我们已经在矢量存储中索引了这些数据，我们将创建一个检索链。该链将接受传入的问题，查找相关文档，然后将这些文档与原始问题一起传递给 LLM 并要求其回答原始问题。

首先，让我们建立一个链，该链接受问题和检索到的文档并生成答案。

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

我们可以使用检索器动态选择最相关的文档并将其传递给给定的问题。

In [33]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [34]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new

In [35]:
response["answer"]

'LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk or create them on the fly, and export test cases from application traces. Additionally, LangSmith offers custom evaluations (both LLM and heuristic based) to score test results, a comparison view to track performance across different configurations, a playground environment for rapid iteration and experimentation, and the ability to capture feedback from users to improve application performance.'

也可以直接传入文档来运行它

In [39]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="LangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled by default.\nOftentimes, it isn’t necessary to look at every single trace. However, when things go wrong (an unexpected end result, infinite agent loop, slower than expected execution, higher than expected token usage), it’s extremely helpful to debug by looking through the application traces. LangSmith gives clear visibility and debugging information at each step of an LLM sequence, making it much easier to identify and root-cause issues.\nWe provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set\u200bWhile many developers still ship an initial version of their application based on “vibe checks”, we’ve seen an increasing number of engineering teams start to adopt a more test driven approach. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.\nThese test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.Comparison View\u200bWhen prototyping different versions of your applications and making changes, it’s important to see whether or not you’ve regressed with respect to your initial test cases.\nOftentimes, changes in the prompt, retrieval strategy, or model choice can have huge implications in responses produced by your application.\nIn order to get a sense for which variant is performing better, it’s useful to be able to view results for different configurations on the same datapoints side-by-side. We’ve invested heavily in a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of your application.Playground\u200bLangSmith provides a playground environment for rapid iteration and experimentation.\nThis allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentimes, this is hooked up to a feedback button in your app), then filter on traces that have a specific feedback tag and score. A common workflow is to filter on traces that receive a poor user feedback score, then drill down into problematic points using the detailed trace view.Annotating Traces\u200bLangSmith also supports sending runs to annotation queues, which allow annotators to closely inspect interesting traces and annotate them with respect to different criteria. Annotators can be PMs, engineers, or even subject matter experts. This allows users to catch regressions across important evaluation criteria.Adding Runs to a Dataset\u200bAs your application progresses through the beta testing phase, it's essential to continue collecting data to refine and improve its performance. LangSmith enables you to add runs as examples to datasets (from both the project page and within an annotation queue), expanding your test coverage on real-world scenarios. This is a key benefit in having your logging system and your evaluation/testing system in the same platform.Production\u200bClosely inspecting key data points, growing benchmarking datasets, annotating traces, and drilling down into important data in trace view are workflows you’ll also want to do once your app hits production.However, especially at the production stage, it’s crucial to get a high-level overview of application performance with respect to latency, cost, and feedback scores. This ensures that it's delivering desirable results at scale.Online evaluations and automations allow you to process and score production traces in near real-time.Additionally, threads provide a seamless way to group traces from a single conversation, making it easier to track the performance of your application across multiple turns.Monitoring and A/B Testing\u200bLangSmith provides monitoring charts that allow you to track key metrics over time. You can expand to view metrics for a given period and drill down into a specific data point to get a trace table for that time period — this is especially handy for debugging production issues.LangSmith also allows for tag and metadata grouping, which allows users to mark different versions of their applications with different identifiers and view how they are performing side-by-side within each chart. This is helpful for A/B testing changes in prompt, model, or retrieval strategy.Automations\u200bAutomations are a powerful feature in LangSmith that allow you to perform actions on traces in near real-time. This can be used to automatically score traces, send them to annotation queues, or send them to datasets.To define an automation, simply provide a filter condition, a sampling rate, and an action to perform. Automations are particularly helpful for processing traces at production scale.Threads\u200bMany LLM applications are multi-turn, meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.")]
})

'LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk or create them on the fly, export test cases from application traces, run custom evaluations, and compare different configurations on the same data points side-by-side. Additionally, LangSmith provides a playground environment for rapid iteration and experimentation, supports beta testing to collect real-world performance data, captures feedback from users, annotates traces with different criteria, adds runs to datasets to expand test coverage, and provides monitoring charts for tracking key metrics over time and A/B testing changes in prompt, model, or retrieval strategy.'

### 深入探索
以上是检索基础知识—要更深入地了解这里提到的所有内容，请参阅[文档](https://python.langchain.com/docs/modules/data_connection/)的这一部分。

## 会话检索链
到目前为止，我们创建的链只能回答单个问题。人们构建的 LLM 应用程序的主要类型之一是聊天机器人。那么我们如何将这个链条变成一个可以回答后续问题的链条呢？
我们仍然可以使用该`create_retrieval_chain`函数，但我们需要更改两件事：

    1. 检索方法现在不应仅适用于最近的输入，而应考虑整个历史记录。
    2. 最终的 LLM 链同样应该考虑整个历史

### 更新检索
为了更新检索，我们将创建一个新链。该链将接受最近的输入 ( `input`) 和对话历史记录 ( `chat_history`) 并使用 LLM 生成搜索查询。


In [46]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [47]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

构建代理时要做的第一件事就是决定它应该有权访问哪些工具。在此示例中，我们将授予代理访问两个工具的权限：

我们刚刚创建的检索器。这将让它轻松回答有关 LangSmith 的问题
一个搜索工具。这将使它能够轻松回答需要最新信息的问题。
首先，让我们为刚刚创建的检索器设置一个工具：
`pip install -U langchain-community tavily-python`

In [48]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

如果您不想设置 API 密钥，则可以跳过创建此工具。


In [53]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

创建我们想要使用的工具的列表

In [54]:
tools = [retriever_tool, search]

可以使用它来获取预定义的提示

In [55]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

我们现在可以调用代理并查看它如何响应！我们可以向它询问有关 LangSmith 的问题：

In [56]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phas

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n2. Debugging: LangSmith provides tracing capabilities to help developers debug issues in LLM applications by offering clear visibility and debugging information at each step of an LLM sequence.\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, allowing for test cases to be uploaded, created on the fly, or exported from application traces.\n4. Comparison View: LangSmith offers a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.\n5. Playground: A playground environment in LangSmit

In [57]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1713250351, 'localtime': '2024-04-15 23:52'}, 'current': {'last_updated_epoch': 1713249900, 'last_updated': '2024-04-15 23:45', 'temp_c': 12.2, 'temp_f': 54.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1021.0, 'pressure_in': 30.16, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 77, 'cloud': 25, 'feelslike_c': 12.7, 'feelslike_f': 54.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 11.4, 'gust_kph': 18.4}}"}, {'url': 'https://www.wunderground.com/hi

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 54.0°F (12.2°C). The wind speed is 6.9 mph (11.2 kph) coming from the west-northwest direction. The humidity is at 77%, and the visibility is 9.0 miles.'}

我们可以和它进行对话：

In [58]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for th

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith is a platform for LLM application development, monitoring, and testing. It supports various workflows that can help in testing your LLM applications, such as prototyping, debugging, creating test sets, comparison views, playground for rapid iteration, beta testing, capturing feedback, annotating traces, adding runs to datasets, production monitoring, A/B testing, automations, and threads for multi-turn applications.\n\nYou can use LangSmith to create datasets, run tests on your LLM applications, compare different versions, gather feedback from users, annotate traces, monitor key metrics, and automate actions on traces in near real-time. This platform provides a comprehensive set of tools to help you test and improve the performance of your LLM applications.'}

现在我们已经成功建立了一个基本代理。我们只涉及了代理的基础知识 - 要深入了解此处提到的所有内容，请参阅[文档](https://python.langchain.com/docs/modules/agents/)的这一部分

## 使用LangServe服务
要为我们的应用程序创建服务器，我们将创建一个`serve.py`文件。这将包含我们为应用程序提供服务的逻辑。它由三部分组成：

1. 我们刚刚在上面构建的链的定义
2. 我们的 FastAPI 应用程序
3. 为链提供服务的路由的定义，由以下命令完成`langserve.add_routes`

